# Building AI Agents with Supabase and Strands 🔷 🧠

Welcome to this tutorial on integrating Supabase with Strands AI agents! In this notebook, you'll learn how to leverage Supabase's powerful backend services through the Model Context Protocol (MCP) to build intelligent agents that can interact with your database, storage, and edge functions.

## What is Supabase?

[Supabase](https://supabase.com/) is an open-source Firebase alternative that provides all the backend services you need to build a product:

- PostgreSQL database
- Authentication
- Instant APIs
- Edge Functions
- Realtime subscriptions
- Storage

## What is Supabase MCP?

Supabase MCP (Model Context Protocol) is Supabase's implementation of a standardized interface that allows AI models to interact directly with Supabase services. It enables Strands AI agents to seamlessly access and manipulate your Supabase database, storage, authentication, and edge functions through natural language instructions. With Supabase MCP, you can build AI agents that can query your PostgreSQL database, manage files in storage, and trigger serverless functions without writing complex integration code for each specific operation.

## What is Strands?

The AWS Strands Agent Framework enables rapid development of AI agents with minimal code. Strands facilitates building highly dynamic agents through natural language, leveraging prompt engineering to generate varied output types and accept diverse natural language inputs seamlessly.

By the end of this tutorial, you'll know how to:
- Connect Strands agents to Supabase using MCP
- Perform database operations with natural language
- Manage edge functions through AI agents
- Handle storage operations with AI assistance

Let's get started!

## Getting Started

Follow these steps to set up:

1. **Sign up** for Supabase at [supabase.com](https://supabase.com/) 
2. **Create** a new project. Name the project "strandsai".

3. **Generate a personal access token** from your [Supabase account settings](https://supabase.com/dashboard/account/tokens).

4. **Paste your token** into the cell below and execute the cell.

In [1]:
# To export your personal access token into a .env file, run the following cell (PLEASE REPLACE WITH YOUR TOKEN):
!echo "SUPABASE_ACCESS_TOKEN=your-personal-access-token" >> .env

Install and import necessary dependencies.

In [ ]:
%pip install strands-agents dotenv supabase --quiet

### Setting Up Your Supabase MCP Client

The code below will instantiate the Supabase MCP client with your personal access token.

In [2]:
import os
import getpass
from dotenv import load_dotenv
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp import StdioServerParameters, stdio_client

# Load environment variables from .env file
load_dotenv()

# Prompt the user to securely input the personal access token if not already set in the environment
if not os.environ.get("SUPABASE_ACCESS_TOKEN"):
    os.environ["SUPABASE_ACCESS_TOKEN"] = getpass.getpass("SUPABASE_ACCESS_TOKEN:\n")

# Initialize the Supabase MCP client
supabase_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="npx", 
            args=[
                "-y",
                "@supabase/mcp-server-supabase@latest",
                "--features=database,storage,functions,docs,debug,account",
                "--access-token", os.getenv("SUPABASE_ACCESS_TOKEN")
            ]
        )
    )
)

## Supabase MCP Architecture

The Supabase MCP integration provides a set of tools that allow AI agents to interact with Supabase services. Here's how it works:

![architecture2.png](assets/architecture2.png)

The flow works as follows:

1. The Strands agent receives a natural language request from the user
2. The agent determines which Supabase service to use based on the request
3. The agent calls the appropriate MCP tool with the necessary parameters
4. The Supabase MCP server executes the request against the Supabase API
5. The Supabase API interacts with the requested service (Database, Storage, or Edge Functions)
6. The results are returned to the agent, which formats them for the user

This architecture allows for seamless integration between AI agents and Supabase services, enabling natural language interactions with your database, storage, and edge functions.

## Creating a Strands Agent with Supabase MCP Tools

Now, let's create a Strands agent that can use the Supabase MCP tools. We'll use the AWS Bedrock Claude model for this example, but you can use any model supported by Strands.

In [3]:
from strands.models import BedrockModel

# Initialize the Bedrock model
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",  # Claude 3.5 Sonnet v2
    # Other model options:
    # model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",  # Claude 3.5 Haiku
    # model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",   # claude 4
    region_name="us-west-2",
)

# Create a system prompt for the agent
system_prompt = """
You are a Supabase expert assistant that helps users manage their Supabase projects, databases, edge functions, and storage.
You have access to Supabase MCP tools that allow you to interact with Supabase services.

When helping users:
1. Always explain what you're doing and why
2. Provide code examples when relevant
3. Suggest best practices for Supabase usage
4. Be security-conscious and avoid exposing sensitive data

You can help with:
- Database operations (SQL queries, table management)
- Edge function deployment and management
- Storage operations (file upload/download, bucket management)
- Project settings and configuration
"""

# Create the agent with Supabase MCP tools
with supabase_client:
    # Get the available tools from the Supabase MCP server
    tools = supabase_client.list_tools_sync()
    
    # Create the agent with the tools and system prompt
    agent = Agent(
        model=bedrock_model,
        system_prompt=system_prompt,
        tools=tools
    )

## Exploring Available Supabase MCP Tools

Let's explore the tools available through the Supabase MCP server. This will help us understand what capabilities our agent has.

In [ ]:
with supabase_client:
    # The tools are already loaded in our agent, so let's ask it directly
    response = agent("What Supabase operations can you help me with? List all the tools and capabilities you have access to.")
    print(response)

## Database Management Examples

Let's start by exploring how our agent can help with database management tasks. We'll ask it to perform various operations on our Supabase database.

In [ ]:
# Example 1: List all tables in the database
with supabase_client:
    response = agent("List all tables in my Supabase strandai project and explain what each one is used for.")
    print(response)

In [ ]:
# Example 2: Create a simple hello world table
with supabase_client:
    response = agent("""
    Create a new table called 'hello_world' with the following columns:
    - id (integer, primary key)
    - message (text, not null)
    - created_at (timestamp with time zone, default to current timestamp)
    """)
    print(response)

In [ ]:
# Example 3: Insert data into the table
with supabase_client:
    response = agent("""
    Insert the following messages into the 'hello_world' table:
    1. Message: 'Hello, World!'
    2. Message: 'Welcome to Supabase!'
    3. Message: 'AI agents are awesome!'
    """)
    print(response)

In [ ]:
# Example 4: Query data from the table
with supabase_client:
    response = agent("Retrieve all messages from the 'hello_world' table and show them with their timestamps.")
    print(response)

## Edge Function Management Examples

Now, let's explore how our agent can help with edge function management. Edge functions are serverless functions that run on Supabase's edge network.

In [ ]:
# Example 1: List all edge functions
with supabase_client:
    response = agent("List all edge functions in my Supabase project.")
    print(response)

In [ ]:
# Example 2: Create a new edge function
with supabase_client:
    response = agent("""
    Create a new edge function called 'hello-world' that returns a JSON response with the following structure:
    {
        "message": "Hello, World!",
        "timestamp": <current timestamp>
    }
    JWT authorization is not needed in this function
    """)
    print(response)

## Storage Management Examples

Finally, let's explore how our agent can help with storage management. Supabase Storage allows you to store and serve files like images, videos, and documents.

In [ ]:
# Example 1: List all storage buckets
with supabase_client:
    response = agent("List all Supabase storage buckets in my Supabase project strandsai.")
    print(response)

## Building a Complete Application: RSS Content Curator

Now that we've explored the individual capabilities of our Supabase-powered agent, let's build a complete RSS content curator system that automatically fetches news, stores it in a database, and sends daily email summaries.

### Prerequisites

Before we start, you'll need to:

1. **Sign up for Resend**: Create a free account at [resend.com](https://resend.com/)
2. **Get your Resend API token**: Go to your Resend dashboard and generate an API key
3. **Add secrets to Supabase Edge Functions**: 
   - Go to your Supabase project dashboard
   - Navigate to **Edge Functions** → **Secrets**
   - Add the following secrets:
     
     **Secret 1 - Resend API Token:**
     - Click **Add new secret**
     - Set **Name**: `RESEND_API_TOKEN`
     - Set **Value**: Your Resend API token (starts with `re_`)
     - Click **Save**
     
     **Secret 2 - Email Address:**
     - Click **Add new secret**
     - Set **Name**: `RECIPIENT_EMAIL`
     - Set **Value**: Your email address (where you want to receive the daily summaries)
     - Click **Save**

These secrets will be securely available to your edge functions for sending emails.

### Build the Complete RSS Curator System

Let's have our agent build the entire RSS content curator system in one go! This will include:
- Database tables for RSS sources and news articles
- RSS fetcher edge function
- Email summary edge function
- Automated cron jobs for hourly RSS fetching and daily email summaries

In [ ]:
with supabase_client:
    response = agent("""
    Build me a complete RSS content curator system in the strandsai project, with the following requirements:
    
    1. Create database tables:
       - rss_sources table (id, name, url, description, active, created_at)
       - news_articles table (id, title, description, link, pub_date, source_id, created_at)
    
    2. Insert BBC News RSS feed (http://feeds.bbci.co.uk/news/rss.xml) as the first source
    
    3. Create 'fetch-rss-content' edge function that:
       - Fetches RSS feeds from active sources
       - Parses XML and extracts articles
       - Only inserts new articles (avoid duplicates)
    
    4. Create 'send-daily-summary' edge function that:
       - Gets articles from last 24 hours
       - Sends HTML email via Resend API
       - Use RESEND_API_TOKEN and RECIPIENT_EMAIL environment variables

    5. Store project URL and Anon key in Supabase Vault

    6. Set up cron jobs to call the edge functions:
       - Hourly RSS fetch (every hour)
       - Daily email summary (8:00 AM daily)
    
    Please build everything and show me the results!
    """)
    print(response)

## What Happens Next?

Congratulations! Your RSS content curator system is now fully operational. Here's what you can expect:

### Automatic Operation
- **Hourly RSS Fetching**: Every hour, your system will automatically fetch the latest news from BBC News RSS feed and store new articles in your database
- **Daily Email Summaries**: Every morning at 8:00 AM, you'll receive a beautifully formatted HTML email with a summary of all news articles from the past 24 hours
- **No Manual Intervention**: The system runs completely automatically using Supabase's cron jobs

### Your Daily Email
The email you receive will include:
- A clean, organized summary of recent news articles
- Article titles, descriptions, and links to read more
- Grouped by RSS source (currently BBC News, but you can add more sources later)
- Professional HTML formatting for easy reading

You should start receiving your first daily summary email tomorrow morning at 8:00 AM!

## Optional: Inspect Your Supabase Dashboard

Want to see what the agent built for you? Here's how to explore your new RSS curator system in the Supabase dashboard:

### 1. Database Tables
- Go to **Database** → **Tables**
- You'll see two new tables:
  - `rss_sources`: Contains your RSS feed sources (currently BBC News)
  - `news_articles`: Will populate with news articles as they're fetched
- Click on each table to see the data and structure

### 2. Edge Functions
- Go to **Edge Functions**
- You'll see two new functions:
  - `fetch-rss-content`: Fetches and parses RSS feeds
  - `send-daily-summary`: Sends your daily email summaries
- Click on each function to view the code and logs

### 3. Cron Jobs
- Go to **Database** → **Cron Jobs** (or check the SQL editor for cron job queries)
- You'll see two scheduled jobs:
  - `hourly-rss-fetch`: Runs every hour to fetch new articles
  - `daily-email-summary`: Runs daily at 8:00 AM to send emails

### 4. Edge Function Secrets
- Go to **Edge Functions** → **Secrets**
- Verify your secrets are properly configured:
  - `RESEND_API_TOKEN`: Your Resend API key
  - `RECIPIENT_EMAIL`: Your email address

### 5. Monitor Activity
- Check **Edge Functions** logs to see function executions
- Monitor the `news_articles` table to see new articles being added
- View function invocation history and any error logs

This gives you full visibility into your automated RSS curator system!

## Conclusion

In this tutorial, we've explored how to integrate Supabase with Strands AI agents using the Model Context Protocol (MCP). We've seen how to:

1. Set up a Supabase MCP client
2. Create a Strands agent with Supabase tools
3. Perform database operations with natural language
4. Manage edge functions through AI agents
5. Handle storage operations with AI assistance
6. Build a complete RSS content curator application

This integration opens up exciting possibilities for building AI-powered applications with Supabase as the backend. You can now create intelligent agents that can interact with your database, storage, and edge functions using natural language, making development faster and more intuitive.

## Next Steps

Here are some ideas for further exploration:

1. Add more RSS sources to your curator system
2. Build a chatbot that can query your Supabase database
3. Create an AI assistant for database schema design
4. Develop a content management system with AI-powered content generation
5. Implement real-time features using Supabase's realtime subscriptions

Happy building with Supabase and Strands! 🔷 🧠